In [2]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time

# 机器学习模块
import xgboost as xgb

# 回测框架
import backtrader as bt
import backtrader.indicators as btind

# 基础函数
import utilsJ

d:\Anaconda\envs\py38\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [14]:
class ml2(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('units', 1),
        ('p_stake', 100),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        # Keep references to lines in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low

        # Keep references to executed order & price
        self.buyprice = 0
        self.sellprice = 0
        self.order = None

        # Add indicators

        self.bollinger = btind.BollingerBands(period=20, devfactor=2)
        self.macdhisto = btind.MACDHisto()
        self.wr = btind.WilliamsR(period=14)

        self.updown = bt.If(self.dataclose(0) > self.dataclose(-1), 1, 0)



    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                self.log('BUY EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash()))

            else:
                self.sellprice = order.executed.price
                self.log('SELL EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash()))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        data_train = xgb.DMatrix(pd.DataFrame({'Bollinger':np.array(self.bollinger.get(ago=-1, size=30)), 
                                               'MACD':np.array(self.macdhisto.get(ago=-1, size=30)),
                                               'wr':np.array(self.wr.get(ago=-1, size=30))}), 
                                 label=np.array(self.updown.get(size=30)))
        data_test = xgb.DMatrix(pd.DataFrame({'Bollinger':np.array(self.bollinger.get(size=1)), 
                                              'MACD':np.array(self.macdhisto.get(size=1)), 
                                              'wr':np.array(self.wr.get(size=1))}))
        params={'booster':'gbtree',
                'nthread':12,
                'objective': 'rank:pairwise',
                'eval_metric':'auc',
                'seed':0,
                'eta': 0.01,
                'gamma':0.1,
                'min_child_weight':1.1,
                'max_depth':5,
                'lambda':10,
                'subsample':0.7,
                'colsample_bytree':0.7,
                'colsample_bylevel':0.7,
                'tree_method':'exact'
                }
        model = xgb.train(params,data_train)
        print(model.predict(data_test)[0])
        #if model.predict(data_test)[0] > 0.51 and self.updown[-1] == 1 and self.updown[-2] == 1 and self.getposition(self.data).size == 0 and self.dataclose[0] > self.bollinger.mid[0]:
        #    cash_ava = self.broker.get_cash()
        #    lots = np.floor(cash_ava / (self.params.p_stake*self.dataclose[0]))*self.params.p_stake
        #    self.order = self.buy(size=lots)
        #elif model.predict(data_test)[0] < 0.5:
        #    self.order = self.close()

    
    #def stop(self):
    #    self.log('Ending Position:%i. Ending Value:%.2f.' %
    #            (self.getposition(self.data).size,
    #            self.cerebro.broker.getvalue()), doprint=False)

In [15]:
if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(ml2, printlog=True) 

    s_date = datetime.datetime.now()- datetime.timedelta(days=385)
    e_date = datetime.datetime.now()

    # Create stock Data Feed
    stock_index = '000001.SH'
    df = utilsJ.index_tushare('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6', 
                        stock_index, s_date, e_date)
    #df = pd.read_csv('C:\\Users\\a\\Desktop\\Backtrader\\Data\\2019\\' + stock_index + '.csv', 
    #                converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
    df['testadd'] = 1
    data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data)

    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(20000)

    # Set commission
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

    # Visualization
    cerebro.plot(iplot=False)

Starting Portfolio Value: 20000.00
0.4837914
0.49373007
0.4913369
0.51058906
0.5139871
0.51782155
0.5191905
0.49646908
0.48473603
0.47490206
0.50104487
0.49906504
0.48988417
0.5146203
0.48867747
0.50333935
0.49409157
0.49695104


d:\Anaconda\envs\py38\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.49400327
0.5018475
0.48143038
0.48620287
0.4788569
0.4964859
0.48795965
0.49230155
0.4941523
0.50639623
0.5018783
0.48794255
0.50175333
0.5023785
0.50849015
0.51749
0.5049873
0.49527043
0.48642132
0.50733364
0.5009927
0.5036606
0.5130356
0.49603692
0.5034438
0.5105916
0.51126665
0.5133843
0.50133157
0.48659867
0.5015455
0.5028107
0.5156559
0.50003344
0.4906981
0.4830789
0.5021655
0.51187235
0.5148627
0.48409018
0.47922269
0.49884474
0.4964201
0.513952
0.5193462
0.5037922
0.48925084
0.49085528
0.498052
0.5131514
0.5119449
0.4991718
0.49195278
0.48077768
0.47995836
0.49235398
0.5286625
0.5059286
0.5155701
0.52463657
0.5071975
0.49135995
0.5081619
0.51652706
0.52174985
0.52675784
0.52969956
0.52053165
0.49978328
0.4891693
0.49120468
0.4945015
0.49499995
0.48906678
0.50479203
0.5119954
0.5114954
0.50306606
0.49982664
0.51894134
0.50534415
0.4928492
0.48097077
0.47916225
0.4706255
0.4915186
0.50496966
0.51568174
0.4936564
0.50385624
0.499379
0.51735127
0.5266302
0.519397
0.5078576
0.51718